# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


####  Run this cell to set up and start your interactive session.


In [58]:
%iam_role arn:aws:iam::212430227630:role/LabRole
%region us-east-1
%number_of_workers 2

%idle_timeout 30
%glue_version 4.0
%worker_type G.1X

Current iam_role is arn:aws:iam::212430227630:role/LabRole
iam_role has been set to arn:aws:iam::212430227630:role/LabRole.
Previous region: us-east-1
Setting new region to: us-east-1
Region is set to: us-east-1
Previous number of workers: 2
Setting new number of workers to: 2
Current idle_timeout is 30 minutes.
idle_timeout has been set to 30 minutes.
Setting Glue version to: 4.0
Previous worker type: G.1X
Setting new worker type to: G.1X


In [61]:
%extra_py_files s3://cryptoengineer/gluejobs-py-modules/load.py, s3://cryptoengineer/gluejobs-py-modules/storage.py
%additional_python_modules yfinance

Extra py files to be included:
s3://cryptoengineer/gluejobs-py-modules/load.py
s3://cryptoengineer/gluejobs-py-modules/storage.py
Additional python modules to be included:
yfinance


In [64]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import sys
import boto3

from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 2
Idle Timeout: 30
Session ID: 75c6b425-343c-4020-890e-1d1a211d5019
Applying the following default arguments:
--glue_kernel_version 1.0.5
--enable-glue-datacatalog true
--extra-py-files s3://cryptoengineer/gluejobs-py-modules/load.py,s3://cryptoengineer/gluejobs-py-modules/storage.py
--additional-python-modules yfinance
Waiting for session 75c6b425-343c-4020-890e-1d1a211d5019 to get into ready status...
Session 75c6b425-343c-4020-890e-1d1a211d5019 has been created.



## HISTORICAL LOAD

### Load modules

In [2]:
from datetime import datetime, timedelta, timezone

import load

### Set AWS storage parameters

In [3]:
BUCKET_NAME = "cryptoengineer"
PREFIX = "datalake/bronze/forex"

## Load job parameters

In [4]:
glue_client = boto3.client("glue")

if '--WORKFLOW_NAME' in sys.argv and '--WORKFLOW_RUN_ID' in sys.argv:
    print("Running in Glue Workflow")
    
    glue_args = getResolvedOptions(
        sys.argv, ['WORKFLOW_NAME', 'WORKFLOW_RUN_ID']
    )
    
    print("Reading the workflow parameters")
    workflow_args = glue_client.get_workflow_run_properties(
        Name=glue_args['WORKFLOW_NAME'], RunId=glue_args['WORKFLOW_RUN_ID']
    )["RunProperties"]

    
    base= workflow_args['base']
    start_date = workflow_args['start_date']
    end_date = workflow_args['end_date']
    symbols = workflow_args['symbols']
    api_key = workflow_args['api_key']

else:
    try:
        print("Running as Job")
        args = getResolvedOptions(sys.argv,
                                  ['JOB_NAME',
                                   'base',
                                   'start_date',
                                   'end_date',
                                   'symbols',
                                   'api_key'])
        base= args['base']
        start_date = args['start_date']
        end_date = args['end_date']
        ##symbols = "EUR,CHF,JPY,CAD,GBP"
        symbols = args['symbols']
        api_key = args['api_key']
    except:
        base= 'USD'
        start_date = '2024-06-01'
        end_date = '2024-07-29'
        ##symbols = "EUR,CHF,JPY,CAD,GBP"
        symbols = "USDEUR"
        api_key= ""


Running as Job


In [ ]:
print("base: ", base)
print("Start Date: ", start_date)
print("End Date: ", end_date)
print("Symbols: ", symbols)
print("API Key: ", api_key)


## Load the historical rates 15min frequency

In [6]:
df = load.load_historical_freq_rates(base=base,
                                      start_date=start_date,
                                      end_date=end_date,
                                      freq='15min',
                                      symbol=symbols,
                                      api_key=api_key,
                                      source='FMP'
)

Year:  2024  Month: 6
https://financialmodelingprep.com/api/v3/historical-chart/15min
Reading month
Lectura API correcta
Leidos  1943
Year:  2024  Month: 7
https://financialmodelingprep.com/api/v3/historical-chart/15min
Reading month
Lectura API correcta
Leidos  4118
2024-07
Leidos  4118
Creating the dataframe


In [7]:
print("Records: ", len(df))

Records:  4118


In [8]:
df.head(5)

                  date     open     low     high    close  volume
0  2024-06-30 23:45:00  0.93002  0.9297  0.93005  0.92996     367
1  2024-06-30 23:30:00  0.92983  0.9296  0.93006  0.93001     317
2  2024-06-30 23:15:00  0.92978  0.9295  0.92991  0.92970     331
3  2024-06-30 23:00:00  0.92978  0.9295  0.92998  0.92950     493
4  2024-06-30 22:45:00  0.93035  0.9295  0.93036  0.92977     408


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4118 entries, 0 to 4117
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    4118 non-null   object 
 1   open    4118 non-null   float64
 2   low     4118 non-null   float64
 3   high    4118 non-null   float64
 4   close   4118 non-null   float64
 5   volume  4118 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 193.2+ KB


### Set the partition columns

In [9]:
freq='15min'
symbol=symbols
source='FMP'

In [10]:
df = load.set_schema_table(df, symbol, source, freq, base)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4118 entries, 0 to 4117
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   datetime       4118 non-null   object             
 1   open           4118 non-null   float64            
 2   low            4118 non-null   float64            
 3   high           4118 non-null   float64            
 4   close          4118 non-null   float64            
 5   volume         4118 non-null   int64              
 6   year           4118 non-null   object             
 7   month          4118 non-null   object             
 8   day            4118 non-null   object             
 9   time           4118 non-null   object             
 10  date           4118 non-null   object             
 11  base_currency  4118 non-null   object             
 12  source         4118 non-null   object             
 13  frequency      4118 non-null   object           

## Save dataframe to raw in parquet format

In [11]:
# Si particionamos por symbol
path=f"s3://{BUCKET_NAME}/{PREFIX}"
print("Path:",path)

Path: s3://cryptoengineer/datalake/raw/forex


In [12]:
(
    spark.createDataFrame(df)
    .repartition("load_date")
    .write
    .format("parquet")
    .mode("append")
    .partitionBy(['load_date'])
    .save(path)
)

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
